## Implementing a simple gradient descent method in Julia

In this blog, we discuss how to implement a simple subgradient descent scheme in Julia using Iterators module. 

#### Background.

Given a convex function $f$, our goal is to solve the following optimization problem: 

\begin{eqnarray*}
\begin{array}{ll}
\textrm{minimize} & f(x)\\
\textrm{subject to} & x\in\mathbf{R}^{n},
\end{array}
\end{eqnarray*}

where $x$ is the decision variable. To solve the problem above, we consider subgradient descent algorithm. The subgradient descent implements the following iteration scheme:

\begin{eqnarray} \label{SGD}
x_{n+1} & = & x_{n}-\gamma_{n}{\nabla f(x_{n})},\qquad (1)
\end{eqnarray}

 where ${\nabla f(x_{n})}$ denotes one subgradient of $f$ evaluated at the iterate $x_{n}$, and $n$ is our iteration counter. As our step size rule, we pick a sequence that is square-summable but not summable, e.g., $\gamma_{n}=1/n$, will do the job. 

Let us load the necessary packages that we are going to use.

In [10]:
using Base.Iterators
using ProximalAlgorithms.IterationTools
using ProximalOperators: Zero
using LinearAlgebra
using Printf

First, we create an `iterable` for our subgradient descent method, which will contain the function $f$ to optimize over and the initial condition $x_0$.

In [47]:
struct GD_iterable{R <: Real, T <: AbstractArray{R}, Tf}
    f::Tf # the objective function
    x0::T # the intial condition
    γ::R # the stepsize
end

Now we define the state of the algorithms is controlled by one iterte $x_n$ and the subgradient ${\nabla{f}(x_n)}$. When the subgradient ${\nabla{f}(x_n)}$ reaches a value that is approximately zero, we terminate our algorithm and take the corresponding $x_n$ as the approximate solution to our opitmization problem.

In [48]:
mutable struct GD_state{T}
    x # iterate x_n
    ∇f_x # one gradient ̃̃∇f(x_n)
    γ # stepsize
    n # iteration counter
end

We need an initialization mechanism for `SGD_state`, which we defnie next.

In [49]:
GD_state(iter::GD_iterable) = GD_state(copy(iter.x), ones(iter.x), 1)

GD_state

Next, we define an `iterate` function that will take $(x_n, {\nabla{f(x_n)}})$, and produce the next state $x_{n+1}$ using $(1)$.

In [50]:
function Base.iterate(iter::GD_iterable, state::GD_state)
    # unpack the current state information
    x_n = state.x
    ∇f_x_n = state.∇f_x
    γ_n = state.γ
    n = state.n
    # compute the next state
    x_n_plus_1 = x_n - γ_n*∇f_x_n
    # now load the computed values in the state
    state.x = x_n_plus_1
    state.∇f_x = gradient(iter.f, x_n_plus_1)
    state.γ = 1/(n+1)
    state.n = n+1
    return state, state
end

Let us write a simple solver, that will take a convex function $f$ and give its optimal solution $x^\star$. 

In [51]:
struct GradientDescent
    γ
    maxit # maximum number of iteration
    tol # tolerance, i.e., if ||∇f(x)|| ≤ tol, we take x to be an optimal solution
    verbose # whether to print information about the iterates
    freq # how often print information about the iterates
    
    # constructor for the structure
    function GradientDescent(; γ = 1, maxit = 1000, tol = 1e-8, verbose = false, freq = 10)
        new(γ, maxit, tol, verbose, freq)
    end
end

In [52]:
# The following function will run the entire loop over the struct GradientDescent

In [53]:
function (solver::GradientDescent)(x0, f) 
    # x0: initial condition
    # f: the function to be optimized over
    
    stop(state::GD_state) = norm(state.res, Inf) <= solver.tol
    disp((it, state)) = @printf("%5d | %.3e\n", it, norm(state.res, Inf))
    
    # time to run the iteration
    iter = GD_iterable(f, x0, solver.γ)
    iter = take(halt(iter, stop), solver.maxit)
    iter = enumerate(iter)
    
    if solver.verbose == true 
        iter = tee(sample(iter, solver.freq), disp)
    end
    
    num_iters, state_final = loop(iter)

    return state_final.x, state_final.∇f_x, state.γ, state.n
end

In [54]:
# Data to test:

In [55]:
using ProximalAlgorithms, ProximalOperators

In [56]:
T = Float64

A = randn(4,5)

b = randn(4)

m, n = size(A)

(4, 5)

In [57]:
# randomized intial point:
x0 = randn(5)

5-element Array{Float64,1}:
  0.34314605308496215
  0.9624123745157233 
 -0.9680974471641903 
 -0.33048683386912975
  0.934294979136089  

In [58]:
f = LeastSquares(A, b)

description : Least squares penalty
domain      : n/a
expression  : n/a
parameters  : n/a

In [59]:
solver = GradientDescent()

GradientDescent(1, 1000, 1.0e-8, false, 10)

In [60]:
solver(x0, f)

MethodError: MethodError: no method matching GD_iterable(::ProximalOperators.LeastSquaresDirect{Float64,Float64,Array{Float64,2},Array{Float64,1},Cholesky{Float64,Array{Float64,2}}}, ::Array{Float64,1}, ::Int64)
Closest candidates are:
  GD_iterable(::Tf, ::T, !Matched::R) where {R<:Real, T<:(AbstractArray{R,N} where N), Tf} at In[47]:2